# Grid Search for non-Clustering Algorithms in Latent Space (+ Comparison to Initial Space)

In [1]:
import os

from multiprocessing.pool import ThreadPool

import ctypes
from ctypes import *

import numpy as np

from tensorflow.keras.models import load_model

from autoencoder import Autoencoder

from helper_funcs import *

import pandas
pandas.set_option('display.max_rows', None)

from params import get_aaf

2024-01-07 23:19:50.889773: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-07 23:19:50.964021: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-07 23:19:50.964911: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-07 23:19:51.960552: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
models = os.listdir('./models/')

dataset = b'MNIST/input.dat'
query   = b'MNIST/query.dat'

model_to_files = {}
for i, model in enumerate(models):
    normalized_dataset = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_normalized_dataset.dat'
    normalized_query   = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_normalized_query.dat'
    encoded_dataset    = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_encoded_dataset.dat'
    encoded_query      = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_encoded_query.dat'

    model_to_files.update({models[i] : [normalized_dataset, normalized_query,
                                        encoded_dataset, encoded_query]})

n = 60000

In [3]:
for model in model_to_files:
    normalized_dataset, normalized_query, encoded_dataset, encoded_query = model_to_files[model]

    model = b'models/' + model.encode()

    # load model
    autoencoder = load_model(model.decode())
    shape = autoencoder.layers[-2].output_shape[1:] # get shape of encoded layer

    # load dataset
    x_train = load_dataset(dataset)
    x_train = x_train.astype('float32') / 255.
    x_test = load_dataset(query)
    x_test = x_test.astype('float32') / 255.
    if len(shape) == 3: # if model type is convolutional
        x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
        x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))
    else:
        x_train = np.reshape(x_train, (len(x_train), 784))
        x_test = np.reshape(x_test, (len(x_test), 784))

    encoded_train = autoencoder.encode(x_train)
    encoded_test = autoencoder.encode(x_test)

    # deflatten encoded datasets
    encoded_train = deflatten_encoded(encoded_train, shape)
    encoded_test = deflatten_encoded(encoded_test, shape)

    # save original datasets normalized
    save_decoded_binary(x_train, normalized_dataset)
    save_decoded_binary(x_test, normalized_query)

    # normalize encoded datasets
    encoded_train = normalize(encoded_train)
    encoded_test = normalize(encoded_test)

    # save encoded datasets
    save_encoded_binary(encoded_train, encoded_dataset)
    save_encoded_binary(encoded_test, encoded_query)

313/313 [==============================] - 1s 3ms/step


In [4]:
best_params_lsh = [7, 4, 1875, 0, 0.6]      # k, L, table_size, query_trick, window
best_params_hypercube = [3, 67, 1000, 0.42] # k, M, probes, window

# k, E, R
best_params_gnns = {'model_conv_46.keras':  [92, 44, 9],
                    'model_conv_12.keras':  [86, 59, 8],
                    'model_conv_19.keras':  [96, 87, 7],
                    'model_dense_26.keras': [70, 69, 9],
                    'model_dense_43.keras': [100, 66, 10], 
                    'model_dense_1.keras':  [64, 58, 10]}

# l
best_params_mrng = {'model_dense_26.keras': [883],
                    'model_conv_46.keras':  [859],
                    'model_conv_19.keras':  [507],
                    'model_dense_1.keras':  [428],
                    'model_dense_43.keras': [818], 
                    'model_conv_12.keras':  [814]}

# l, m, k, lq
best_params_nsg = {'model_dense_26.keras': [987, 52, 69, 667],
                   'model_conv_46.keras':  [987, 43, 87, 476],
                   'model_dense_43.keras': [960, 102, 86, 768],
                   'model_dense_1.keras':  [996, 11, 98, 657],
                   'model_conv_12.keras':  [635, 197, 50, 907],
                   'model_conv_19.keras':  [663, 99, 41, 533]}

rows = []

In [5]:
def run_brute_force(model):
    normalized_dataset, normalized_query, encoded_dataset, encoded_query = model_to_files[model]

    conf = {
        'model': b'BRUTE',
        'vals': [],
        'dataset': normalized_dataset,
        'query': normalized_query,
        'encoded_query': encoded_query,
        'encoded_dataset': encoded_dataset
    }

    aaf_lat_init, average_time = get_aaf(100, conf)

    print("Average time           :", average_time.value)
    print("AAF (latent to initial):", aaf_lat_init.value)
    print("-------------------------")

    rows.append([model, 'BRUTE', average_time.value, aaf_lat_init.value])

def run_lsh(model):
    normalized_dataset, normalized_query, encoded_dataset, encoded_query = model_to_files[model]
    
    conf = {
        'model': b'LSH',
        'vals': best_params_lsh[:-1],
        'window': best_params_lsh[-1],
        'dataset': normalized_dataset,
        'query': normalized_query,
        'encoded_query': encoded_query,
        'encoded_dataset': encoded_dataset
    }

    aaf_lat_init, average_time = get_aaf(100, conf)

    print("Average time           :", average_time.value)
    print("AAF (latent to initial):", aaf_lat_init.value)
    print("-------------------------")

    rows.append([model, 'LSH', average_time.value, aaf_lat_init.value])

def run_hypercube(model):
    normalized_dataset, normalized_query, encoded_dataset, encoded_query = model_to_files[model]
    
    conf = {
        'model': b'CUBE',
        'vals': best_params_hypercube[:-1],
        'window': best_params_hypercube[-1],
        'dataset': normalized_dataset,
        'query': normalized_query,
        'encoded_query': encoded_query,
        'encoded_dataset': encoded_dataset
    }

    aaf_lat_init, average_time = get_aaf(100, conf)

    print("Average time           :", average_time.value)
    print("AAF (latent to initial):", aaf_lat_init.value)
    print("-------------------------")

    rows.append([model, 'CUBE', average_time.value, aaf_lat_init.value])

def run_gnns(model):
    normalized_dataset, normalized_query, encoded_dataset, encoded_query = model_to_files[model]
    
    conf = {
        'model': b'GNNS',
        'vals': best_params_gnns[model],
        'dataset': normalized_dataset,
        'query': normalized_query,
        'encoded_query': encoded_query,
        'encoded_dataset': encoded_dataset
    }

    aaf_lat_init, average_time = get_aaf(100, conf)

    print("Average time           :", average_time.value)
    print("AAF (latent to initial):", aaf_lat_init.value)
    print("-------------------------")

    rows.append([model, 'GNNS', average_time.value, aaf_lat_init.value])

def run_mrng(model):
    normalized_dataset, normalized_query, encoded_dataset, encoded_query = model_to_files[model]
    
    conf = {
        'model': b'MRNG',
        'vals': best_params_mrng[model],
        'dataset': normalized_dataset,
        'query': normalized_query,
        'encoded_query': encoded_query,
        'encoded_dataset': encoded_dataset
    }

    aaf_lat_init, average_time = get_aaf(100, conf)

    print("Average time           :", average_time.value)
    print("AAF (latent to initial):", aaf_lat_init.value)
    print("-------------------------")

    rows.append([model, 'MRNG', average_time.value, aaf_lat_init.value])

def run_nsg(model):
    normalized_dataset, normalized_query, encoded_dataset, encoded_query = model_to_files[model]
    
    conf = {
        'model': b'NSG',
        'vals': best_params_nsg[model],
        'dataset': normalized_dataset,
        'query': normalized_query,
        'encoded_query': encoded_query,
        'encoded_dataset': encoded_dataset
    }

    aaf_lat_init, average_time = get_aaf(100, conf)

    print("Average time           :", average_time.value)
    print("AAF (latent to initial):", aaf_lat_init.value)
    print("-------------------------")

    rows.append([model, 'NSG', average_time.value, aaf_lat_init.value])

In [6]:
algorithms_to_functions = {'BRUTE': run_brute_force,
                           'LSH'  : run_lsh,
                           'CUBE' : run_hypercube,
                           'GNNS' : run_gnns,
                           'MRNG' : run_mrng,
                           'NSG'  : run_nsg}

pool = ThreadPool(processes=4)

for model in models:
    for algorithm in algorithms_to_functions:
        pool.apply_async(algorithms_to_functions[algorithm], (model,))

pool.close()
pool.join()

Average time           : 0.004130890000000001
AAF (latent to initial): 1.1195060448127099
-------------------------
Average time           : 0.024650560000000005
AAF (latent to initial): 1.1137714166683963
-------------------------
Average time           : 5.7950000000000026e-05
AAF (latent to initial): 1.8247506709576928
-------------------------
Average time           : 0.039317059999999994
AAF (latent to initial): 1.0660772379744767
-------------------------
Average time           : 0.005287209999999999
AAF (latent to initial): 1.1154923799122258
-------------------------
Average time           : 0.008201419999999997
AAF (latent to initial): 1.1064807515044284
-------------------------
Average time           : 0.0001440899999999999
AAF (latent to initial): 1.7310843094549349
-------------------------
Average time           : 0.008482249999999998
AAF (latent to initial): 1.077374530672516
-------------------------
Average time           : 0.003189710000000001
AAF (latent to initial):

In [7]:
col_models, col_algorithms, col_average_time, col_aaf_lat_init = [], [], [], []

for row in rows:
    model, algorithm, average_time, aaf_lat_init = row

    col_models.append(model)
    col_algorithms.append(algorithm)
    col_average_time.append(average_time)
    col_aaf_lat_init.append(aaf_lat_init)

col_dict = {'model': col_models, 'algorithm': col_algorithms,
            'average time': col_average_time, 'AAF (latent to initial)': col_aaf_lat_init}

df = pandas.DataFrame(data=col_dict)
df

,model,algorithm,average time,AAF (latent to initial)
0,model_conv_46.keras,LSH,0.004131,1.119506
1,model_conv_46.keras,BRUTE,0.024651,1.113771
2,model_conv_46.keras,CUBE,0.000058,1.824751
3,model_conv_12.keras,BRUTE,0.039317,1.066077
4,model_conv_46.keras,GNNS,0.005287,1.115492
5,model_conv_12.keras,LSH,0.008201,1.106481
6,model_conv_12.keras,CUBE,0.000144,1.731084
7,model_conv_12.keras,GNNS,0.008482,1.077375
8,model_conv_46.keras,NSG,0.003190,1.120786
9,model_dense_43.keras,BRUTE,0.034395,1.109476


In [8]:
df_sorted = df.copy(deep=True)
df_sorted = df_sorted.sort_values(by='model', ascending=True)
df_sorted = df_sorted.reset_index(drop=True)
df_sorted

,model,algorithm,average time,AAF (latent to initial)
0,model_conv_12.keras,BRUTE,0.039317,1.066077
1,model_conv_12.keras,MRNG,0.006476,1.066077
2,model_conv_12.keras,LSH,0.008201,1.106481
3,model_conv_12.keras,CUBE,0.000144,1.731084
4,model_conv_12.keras,GNNS,0.008482,1.077375
5,model_conv_12.keras,NSG,0.009167,1.069967
6,model_conv_19.keras,MRNG,0.002589,1.184332
7,model_conv_19.keras,NSG,0.004417,1.201835
8,model_conv_19.keras,GNNS,0.008461,1.196865
9,model_conv_19.keras,CUBE,0.000195,1.786995


In [9]:
df_sorted = df.copy(deep=True)
df_sorted = df_sorted.sort_values(by='algorithm', ascending=True)
df_sorted = df_sorted.reset_index(drop=True)
df_sorted

,model,algorithm,average time,AAF (latent to initial)
0,model_dense_26.keras,BRUTE,0.029073,1.098670
1,model_conv_46.keras,BRUTE,0.024651,1.113771
2,model_conv_12.keras,BRUTE,0.039317,1.066077
3,model_dense_1.keras,BRUTE,0.048395,1.052927
4,model_dense_43.keras,BRUTE,0.034395,1.109476
5,model_conv_19.keras,BRUTE,0.038094,1.184543
6,model_conv_46.keras,CUBE,0.000058,1.824751
7,model_dense_1.keras,CUBE,0.000192,1.747885
8,model_conv_12.keras,CUBE,0.000144,1.731084
9,model_conv_19.keras,CUBE,0.000195,1.786995


# Results

## Best algorithm for each latent space

| model | latent dimension | best (AAF) | best (average query time) |
|:-----:|:----------------:|:----:|:------------------:|
| `model_conv_12.keras`    | $25$ | MRNG  | CUBE |
| `model_conv_19.keras`    | $32$ | MRNG  | CUBE |
| `model_conv_46.keras`    | $18$ | MRNG  | CUBE |
| `model_dense_1.keras`    | $46$ | NSG   | CUBE |
| `model_dense_26.keras`   | $23$ | MRNG  | CUBE |
| `model_dense_43.keras`   | $33$ | BRUTE | CUBE |

## Worst algorithm for each latent space

| model | latent dimension | worst (AAF) | worst (average query time) |
|:-----:|:----------------:|:----:|:------------------:|
| `model_conv_12.keras`    | $25$ | CUBE  | BRUTE |
| `model_conv_19.keras`    | $32$ | CUBE  | BRUTE |
| `model_conv_46.keras`    | $18$ | CUBE  | BRUTE |
| `model_dense_1.keras`    | $46$ | CUBE  | BRUTE |
| `model_dense_26.keras`   | $23$ | CUBE  | BRUTE |
| `model_dense_43.keras`   | $33$ | CUBE  | BRUTE |

## Average construction time in initial and latent space

|      | initial       | latent    |
|:----:|:-------------:|:---------:|
| LSH  | $1$ min       | $1-3$ sec |
| CUBE | $30$ sec      | $4-7$ sec |
| GNNS | $50-70$ sec   | $30-70$ sec |
| MRNG | $8$ h         | $20-30$ min |
| NSG  | $15-40$ min   | $10-20$ min |

# Conclusions

The MRNG algorithm is the best for the vast majority of latent spaces in terms of AAF. In the latent space of the highest dimension, the NSG algorithm was the one that lead to the best AAF. This small improvement in AAF compared to brute force can be explained by the model loss of the autoencoder, as there is no 1-1 correspondence between the latent and the initial space.

On the other hand, the Hypercube algorithm is the best in terms of average query time for all latent spaces but at the expense of yielding the worst AAF of all the algorithms. However, the AAF values it produces are not that large, as they are always under $2$. This happens because of the small number of points checked compared to the other algorithms ($M$ parameter).

The average query time for the brute-force algorithm was reduced from $0.5$ sec to under $0.05$ sec when the latent space encodings were introduced.

The brute force algorithm is the worst in terms of average query time for all latent spaces, regardless of the dimension of the encodings.
<br></br>

In lower dimensions, approximate algorithms are not as time-efficient in higher dimensions. In dimensions less than $50$, we see an improvement of $1-2$ orders of magnitude in the average query time, while in higher dimensions, the improvement is $3-4$ orders of magnitude.
<br></br>

All algorithms yield AAF values very close to $1$, except the Hypercube algorithm which has higher AAF values, closer to $2$, but still acceptable.
<br></br>

The average construction time of all algorithms in the latent space has decreased. However, the largest difference in construction times was recorded in MRNG, to the extent that allows us to take advantage of the dimensionality reduction. Instead of spending $8$ hours for the construction of the graph in the initial space, we train an autoencoder in approximately $1-2$ mins (given the best model parameters and architecture) and then run the MRNG algorithm in the latent space for about $20-30$ mins instead, with very little difference in the final results in terms of AAF.

The AAF in projected latent space is roughly $20\%$ lower than the AAF in initial space ($1.2$ in most cases vs $<1.5$), but the average query times are not changing dramatically (~$0.01$ vs ~$0.001$) for most algorithms. However, we gain a lot in terms of construction time (~$2$ mins to construct the NN autoencoder).